# Сегментация изображений

## Задача сегментации

Сегментация изображения — задача поиска групп пикселей, каждая из которых характеризует один смысловой объект.

Результатом сегментации, как правило, является маска, которая отделяет различные типы объектов.

<img src="images_seg/segtask0.png" alt="Image segmentation" style="width: 800px; height: 400px;">


Отличием от задачи классификации является то, что результат относится не ко всему изображению, а к его различным фрагментам.

## Подготовка данных

Соответственно задаче, нам понадобится разметка набора данных. Эта разметка должна обеспечить идентификацию местоположения объекта на изображении. Задача очень похожа на задачу классификации, только надо для каждого пикселя сказать, где у нас какой объект. При этом можно провести аналоги между бинарной и многоклассовой классификацией:
* определить, где объект, а где фон
* определить положение нескольких разнотипных объектов на изображении

Чтобы обучить нейросетевую модель, необходимо разметить данные. Разметка заключается в построении маски, похожей на показанную на рисунке ниже. Маска позволяет выделить интересующие объекты.

<img src="images_seg/segtask1.png" alt="Image segmentation" style="width: 800px; height: 400px;">

Если рассмотреть маску ближе, то мы увидим, что каждой точке соответствует значение, обозначающее класс:

<img src="images_seg/mask.png" alt="Image segmentation" style="width: 800px; height: 400px;">

## Сегментация с помощью полосвязной свёрточной нейросети

Для сегментации применяются различные свёрточные нейронные сети:
* FCNN - полносвязная CNN
* U-Net
* ResNet
* R-FCN
* Mask R-CNN
* и др.

## Архитектура U-Net

## Задание